<a href="https://colab.research.google.com/github/barisgg/blogz/blob/master/Solutions_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="#de3023"><h1><b>REMINDER MAKE A COPY OF THIS NOTEBOOK, DO NOT EDIT</b></h1></font>

#Conscientious Cars 2: Convolutional Neural Nets

Welcome back to CC: ConscientiousCars! Today, we'll be improving on our system for distinguishing dogs from roads.

In [ ]:
#@title Run this to load some packages and data! { display-mode: "form" }
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from collections import Counter
import keras.api._v2.keras as keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, InputLayer
from keras.layers import Activation, MaxPooling2D, Dropout, Flatten, Reshape
from keras.utils.np_utils import to_categorical
!pip install -q git+https://github.com/rdk2132/scikeras # workaround for scikeras deprecation
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# Quiet deprecation warnings
import warnings
warnings.filterwarnings("ignore")

def categorical_to_numpy(labels_in):
  labels = []
  for label in labels_in:
    if label == 'dog':
      labels.append(np.array([1, 0]))
    else:
      labels.append(np.array([0, 1]))
  return np.array(labels)

def load_data():
  # import the data from the Cloud
  !wget -q --show-progress https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%201%20-%205/Session%204%20_%205%20-%20Neural%20Networks%20_%20CNN/dogs_v_roads

  # initialize our input and output variables
  data_dict = pickle.load(open( "dogs_v_roads", "rb" ));
  data   = data_dict['data']
  labels = data_dict['labels']

  return data, labels


def plot_one_image(data, labels, img_idx):
  from google.colab.patches import cv2_imshow
  import cv2
  import matplotlib.pyplot as plt
  my_img   = data[img_idx, :].squeeze().reshape([32,32,3]).copy()
  my_label = labels[img_idx]
  print('label: %s'%my_label)
  plt.imshow(my_img)
  plt.show()

def CNNClassifier(num_epochs=2, layers=1, dropout=0.15):
  def create_model():
    model = Sequential()
    model.add(Reshape((32, 32, 3)))

    for i in range(layers):
      model.add(Conv2D(32, (3, 3), padding='same'))
      model.add(Activation('relu'))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    return model
  # initiate RMSprop optimizer
  opt = keras.optimizers.legacy.RMSprop(learning_rate=0.0001, decay=1e-6)
  return KerasClassifier(model=create_model, optimizer=opt, loss='categorical_crossentropy', epochs=num_epochs, batch_size=10, verbose=2, validation_batch_size=10, validation_split=.2, metrics=['accuracy'])


def plot_acc(history, ax = None, xlabel = 'Epoch #'):

    if hasattr(history, 'history_'):
      history = history.history_
    else:
      history = history.history
    history.update({'epoch':list(range(len(history['val_accuracy'])))})
    history = pd.DataFrame.from_dict(history)

    best_epoch = history.sort_values(by = 'val_accuracy', ascending = False).iloc[0]['epoch']

    if not ax:
      f, ax = plt.subplots(1,1)
    sns.lineplot(x = 'epoch', y = 'val_accuracy', data = history, label = 'Validation', ax = ax)
    sns.lineplot(x = 'epoch', y = 'accuracy', data = history, label = 'Training', ax = ax)
    ax.axhline(0.5, linestyle = '--',color='red', label = 'Chance')
    ax.axvline(x = best_epoch, linestyle = '--', color = 'green', label = 'Best Epoch')
    ax.legend(loc = 7)
    ax.set_ylim([0.4, 1])

    ax.set_xlabel(xlabel)
    ax.set_ylabel('Accuracy (Fraction)')

    plt.show()

def model_to_string(model):
    import re
    stringlist = []
    model.summary(print_fn=lambda x: stringlist.append(x))
    sms = "\n".join(stringlist)
    sms = re.sub('_\d\d\d','', sms)
    sms = re.sub('_\d\d','', sms)
    sms = re.sub('_\d','', sms)
    return sms

In this notebook, we will:

- Use a pre-built CNN function to classify roads vs. dogs.
- Build neural networks from scratch in Keras.
- Experiment with building CNN models from scratch in Keras.
- (Advanced, Optional) Build CNN models for distinguishing cats from dogs, and even experiment with implementing a famous architecture!


<font color=darkorange>**Change Hardware Accelerator to GPU to train faster (Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU)**

#Loading in Data

Once again, let's load in our dog/road dataset and create our training and test set. **What's the shape of each dataset? Why?**

In [ ]:
# Load our data
data_raw, labels_raw = load_data()
data = data_raw.astype(float)
labels = categorical_to_numpy(labels_raw)
inputs_train, inputs_test, labels_train, labels_test = model_selection.train_test_split(data, labels, test_size=0.2, random_state=1)

#Find the shape of our dataset!
### YOUR CODE HERE

### END CODE

In [ ]:
#@title Instructor Solution
# load our data
data_raw, labels_raw = load_data()
data = data_raw.astype(float)
labels = categorical_to_numpy(labels_raw)
inputs_train, inputs_test, labels_train, labels_test = model_selection.train_test_split(data, labels, test_size=0.2, random_state=1)

#Find the shape of our dataset!

## YOUR CODE HERE
print(data.shape)
print(data)

Use the cell below as a reminder of what the data looks like.

In [ ]:
plot_one_image(data_raw, labels_raw, 300) # change this number

# Models for Vision: Convolutional Neural Networks

###Exercise: Exploring Hyperparameters ✍️

As you know, there is a famous type of neural network known as convolutional neural networks (CNNs). These types of neural networks work particularly well on problems to do with computer vision. Let's try one out!

To load up a simple CNN, just run:

`cnn = CNNClassifier(num_epochs, layers, dropout)`

Work with your instructors to review what each parameter means and how it affects the model! The value for **dropout** is a float between 0 and 1 that represents the probability the weight for a neuron in the layer is set to 0 during training time. Each neuron in the layer is evaluated as such, which can help prevent overfitting.

**Try different values of num_epochs, layers, and dropout so that you get the best possible accuracy on the test set using `model.score()`**!

In [ ]:
cnn = CNNClassifier(5, 2, 0.5)
cnn.fit(inputs_train, labels_train)
print(cnn.score(inputs_test, labels_test))

In [ ]:
#@title Instructor Solution { display-mode: "form" }

# Create and train our multi layer perceptron model
cnn = CNNClassifier(num_epochs=14, layers=6, dropout=0.5)
cnn.fit(inputs_train, labels_train)

# Print the score on the testing data
print("CNN Testing Set Score:")
print(cnn.score(inputs_test, labels_test))

**How well did your neural network perform?**

CNNs typically perform better than fully-connected neural networks on vision problems, but, as before, they aren't always consistent. They are also sensitive to a number of parameters.

## Training and Validation Curves

An important aspect of training neural networks is to prevent overfitting. **How would we recognize overfitting?**

In the first line of code below, we first **fit** the model on the training data and pass in some validation (or test) data to evaluate it. We call it the **history** because we want to retain information about the accuracy at each epoch.

In the second line we plot the history so that we can compare the training and validation accuracies.  

```
history = model.fit(inputs_train, labels_train, validation_data=(inputs_test, labels_test))
plot_acc(history)
```

###Exercise: Plotting a Training vs. Validation Curve For Our CNN ✍️

**After how many epochs does the model begin to overfit? How does this vary as you vary the number of hidden layers and dropout?** Overfitting occurs when the validation accuracy starts to drop below the training accuracy.

In [ ]:
### YOUR CODE HERE

### END CODE

In [ ]:
#@title Instructor Solution { display-mode: "form" }
cnn = CNNClassifier(num_epochs=14, layers=6, dropout=0.5)

history = cnn.fit(inputs_train, labels_train,
                  validation_data=(inputs_test, labels_test))
plot_acc(history)

In [ ]:
# Print the score on the testing data
print("CNN Testing Set Score:")
print(cnn.score(inputs_test, labels_test))

# Building Neural Networks from Scratch in Keras

So far, we've used helper functions which pre-build Keras neural network models. Now, we will build them on our own!

Let's start with a "toy example": a tiny neural network with just three numerical inputs.


###Exercise: Building a Simple Neural Network Using Keras! ✍️




We're going to build this model:

![](http://cs231n.github.io/assets/nn1/neural_net.jpeg)

This network can be described as:
* Input Layer: 3 neurons
* Layer 1 (Hidden): 4 neurons that are activated by `'relu'`
* Layer 2 (Output): 2 neurons that are activated by `'softmax'`


We also want to compile the model with
`loss = 'categorical_crossentropy'`

What does this represent? Here's one way to interpret it:
* This model classifies animals as "cat" or "dog"
* Our three inputs are height, weight, and age
* Our ouputs represent "probability of cat" and "probability of dog"
* Because this is a toy example, we aren't actually training the model here - just using randomly initialized weights! We will train later models in this notebook.

Try filling in the blanks below and walking through each line! **If you want a hint or more details, check out the optional reference below.**

In [ ]:
# Fill in the blanks with your group!
### YOUR CODE HERE:
model_1 = Sequential()
model_1.add(InputLayer(input_shape=(____,)))
model_1.add(Dense(____, activation = '____'))
model_1.add(Dense(____, activation = '____'))
model_1.compile(loss='____',
                optimizer = 'adam',
                metrics = ['accuracy'])
model_1.predict([[14,18,5]]) #Try any input! This represents an animal of height 14, weight 18, and age 5.
### END CODE

In [ ]:
#@title Instructor Solution { display-mode: "form" }
# Fill in the blanks with your group!
### YOUR CODE HERE:
model_1 = Sequential()
model_1.add(InputLayer(input_shape=(3,)))
model_1.add(Dense(4, activation = 'relu'))
model_1.add(Dense(2, activation = 'softmax'))
model_1.compile(loss='categorical_crossentropy',
optimizer = 'adam',
metrics = ['accuracy'])
model_1.predict([[14,18,5]]) #Try any input! This represents an animal of height 14, weight 18, and age 5.

### END CODE

**Discuss:** How would you interpret this output? Does our (untrained) network classify this as a cat or a dog?

In [ ]:
# @title Instructor Solution { display-mode: "form" }
# The first value is the probability of the animal being a cat,
# the second is the probability of it being a dog.

####**Optional Reference**

Here's some information about each step of the process. **You don't need to read through all this - check it as a reference if needed!**

**1. Specify model**

```
model = Sequential()
```
In this line of code, we build our network where the information flows from LEFT to RIGHT through the network in ONE DIRECTION as opposed to multiple directions. Neurons on the right never pass informations to neurons on the left of it.


**2. Add layers to the network**
```
model.add(Dense(4, activation = 'sigmoid'))
```
In this code, we add a layer of neurons to our network.

This layers consists of 4 neurons. Each neuron is DENSE and connects to all of the previous layer's inputs and all of the subsequent layers outputs. We specify that there are 3 inputs here.

We also specify what kind of output the neuron will give. If you want the neuron to output a number between 0 and 1 (like a probability!) you would use `'softmax'` or `'sigmoid'`. If you want the neuron to output any number, you can use `'linear'`! You'll also often see `'relu'`, which is when a neuron will only output positive numbers.

```
model.add(Dense(1, activation = 'linear'))
```
This code adds ANOTHER layer to the network that has 1 neuron. This one neuron is used to predict a continuous value!

**3. Turn the model on by compiling it**

After having built the network, we want to train and use it, so we have to 'compile' it to prepare. We have to specify at the very least: a loss (how the model measures the quality of its weights), an optimizer (which adjusts the weights), and a metric (how to evaluate our results). Here are some common choices:
```
model.compile(loss='mean_squared_error',
optimizer = 'adam',
metrics = ['mean_squared_error'])
  ```

Once we've created our network, we can use it very simply! Just like we did with sklearn, we define our input data (x), the true predictions from that data (y), and then train our model with `fit`.

```
model.fit(x, y)
```

To use the model, you can use it to predict something with:
```
y = model.predict(x)
```

You can actually use the model before you even train it! It just won't perform very well.

###(Optional) Exercise: Building a Multi-layer Neural Net Using Keras ✍️

![](http://cs231n.github.io/assets/nn1/neural_net2.jpeg)

Let's try another, bigger example!

Here, we are predicting a house price: regression! Our inputs could be "year the house was built", "home square footage", and "lot square footage", while our output is price (in thousands of dollars).

* Input Layer: 3 neurons

* Layer 1: 4 neurons that are activated by `'relu' `

* Layer 2: 4 neurons that are activated by `'relu'`

* Layer 3 (out): 1 neuron that is activated by `'relu'`

Compile the model with
`'mean_squared_error'` as both loss and metric, and try making a prediction for some made-up data.


In [ ]:
### YOUR CODE HERE

### END CODE

In [ ]:
#@title Instructor Solution { display-mode: "form" }

### YOUR CODE HERE
model_2 = Sequential()
model_2.add(InputLayer(input_shape=(3,)))
model_2.add(Dense(4, activation = 'relu'))
model_2.add(Dense(4, activation = 'relu'))
model_2.add(Dense(1, activation = 'relu'))
model_2.compile(loss='mean_squared_error',
optimizer = 'adam',
metrics = ['mean_squared_error'])

model_2.predict([[2012, 1000, 1400]])
#This house was built in 2012, is 1000 square feet, and has 1400 square feet of land.

### END CODE

###(Optional) Exercise: Dogs vs. Roads Using Keras

Let's try an even bigger example! Here, we are going to distinguish between images of dogs and roads once again.

* Input Layer: 3072 dimensions (32 pixels x 32 pixels x 3 color channels)

* Layer 1: 32 neurons that are activated by `'relu' `and take in 3072 inputs.

* Layer 2: 16 neurons that are activated by `'relu'`

* Layer 3 (out): 2 neurons that are activated by `'softmax'`

Compile the model with
`loss = 'categorical_crossentropy'`, and try making predictions on `inputs_train`!

Once again, we are not actually training this model - so the predictions won't be any good. Soon we will create a CNN, which we will train!

In [ ]:
### YOUR CODE HERE

### END CODE

In [ ]:
#@title Instructor Solution
### YOUR CODE HERE:
model_3 = Sequential()
model_3.add(InputLayer(input_shape=(3072,)))
model_3.add(Dense(32, activation = 'relu'))
model_3.add(Dense(16, activation = 'relu'))
model_3.add(Dense(2, activation = 'softmax'))
model_3.compile(loss='categorical_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])

model_3.predict(inputs_train[:1])
### END CODE

##Exercise: Building a CNN Using Keras! ✍️

Now that we know how to build simple neural networks in Keras, let's build a CNN! The CNN will perform well on our data set of car and road images.

Below is Keras code for a CNN. It will run as-is on the conscientious cars dataset. However, the performance is suboptimal. Add more layers and change the neural network hyperparameters so that the performance will be better. **Can you get the train and validation accuracy to both be higher than 95%?**

The Keras core layer API may be a useful reference: https://keras.io/layers/core/

In particular and in addition to adding more of the existing convolutional layers and activations, consider using the following layers after a convolution + activation:

`Dropout(N)`

`MaxPooling2D(pool_size=(N, N))`


In [ ]:
model = Sequential()
model.add(Reshape((32, 32, 3)))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

###
###
### TODO: ADD MORE LAYERS HERE!!!!!
###
###

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.legacy.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# Train the CNN and plot accuracy.
history = model.fit(inputs_train, labels_train, \
                    validation_data=(inputs_test, labels_test), \
                    epochs=70)
plot_acc(history)


In [ ]:
#@title Instructor Notes + Solution { display-mode: "form" }

"""
Depending on how the students define their model,
the validation accuracy may be significantly higher
than the training accuracy at some epochs. This can be
attributed to:
(1) the fact that Keras turns off regularization
(like dropout) during test time and
(2) that Keras reports the validation accuracy at the end
of each epoch, but reports the average training accuracy over
all the batches in each epoch -- this may be especially
significant in the first epoch.

See [Keras FAQ](https://keras.io/getting_started/faq/#why-is-my-training-loss-much-higher-than-my-testing-loss) for more
"""

model = Sequential()
model.add(Reshape((32, 32, 3)))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.legacy.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# Train the CNN and plot accuracy.
history = model.fit(inputs_train, labels_train, \
                    validation_data=(inputs_test, labels_test), \
                    epochs=70)
plot_acc(history)



**What interesting observations** do you make from the graph? How many epochs should you train for?

We can also print out the structure of our model. What do the parts of the summary mean?

In [ ]:
model.summary()

#Advanced: Cats vs. Dogs with CNN

So far, we've trained a CNN to distinguish between small images of roads and small images of dogs. It's more challenging and time-consuming to train CNNs for bigger images or harder tasks, like distinguishing dogs from cats (which look a lot more like dogs than roads do!)

In this exercise, you'll adapt your previous model to classify large images of dogs vs. cats, and then try implementing a famous CNN architecture. Along the way, you'll deal with some of the debugging that machine learning engineers often have to handle.

In [ ]:
#@title Run this to load cat and dog data. { display-mode: "form" }

#Code here from https://colab.research.google.com/github/google/eng-edu/blob/master/ml/pc/exercises/image_classification_part1.ipynb#scrollTo=4PIP1rkmeAYS

import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab.patches import cv2_imshow
import cv2
import matplotlib.pyplot as plt

try:
  road_model = model
  road_saved = True
except NameError:
  road_saved = False

IMG_SHAPE  = 150  # Our training data consists of images with width of 150 pixels and height of 150 pixels
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_dir = tf.keras.utils.get_file('cats_and_dogs_filterted.zip', origin=_URL, extract=True)
base_dir = os.path.join(os.path.dirname(zip_dir), 'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')  # directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # directory with our validation dog pictures
train_image_generator      = ImageDataGenerator()  # Generator for our training data
validation_image_generator = ImageDataGenerator()  # Generator for our validation data
train_data = train_image_generator.flow_from_directory(batch_size=2000,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_SHAPE,IMG_SHAPE), #(150,150)
                                                           class_mode='binary').next()
val_data = validation_image_generator.flow_from_directory(batch_size=1000,
                                                              directory=validation_dir,
                                                              shuffle=False,
                                                              target_size=(IMG_SHAPE,IMG_SHAPE), #(150,150)

                                                              class_mode='binary').next()
cd_train_inputs, cd_train_labels = train_data
cd_test_inputs, cd_test_labels = val_data

**Run the code below to see the dimensions of our training and validation data. What does each number mean? What is different than our previous dataset?**

In [ ]:
print(cd_train_inputs.shape)
print(cd_train_labels.shape)
print(cd_test_inputs.shape)
print(cd_test_labels.shape)

**Run this code to see a random image from our training data (different each time).**

In [ ]:
index = np.random.randint(len(cd_train_inputs))
plt.imshow(cd_train_inputs[index]/255)
plt.show()
print("Label:",cd_train_labels[index])

**By adapting code from the previous exercise, build, train, and test a CNN to classify cats vs. dogs.**
**Hints:**
*   Use `print(model.summary())` for a useful visualization of your model's architecture. Compare the summary of your cat/road and cat/dog classifiers.
*  Substitute the names of the new datasets.
*  Get a "first try" working by making small adjustments to a previous model before trying to optimize the accuracy. You can temporarily comment out layers as you figure things out.
*  The outputs have different shapes betweeen the two datasets. What do you need to change? (You will get an ValueError that suggests how to transform the output to a one-hot encoding. Hint: you may want to look into how to use the `to_categorical()` function)
*  If you run out of memory, restart the notebook and/or use your knowledge of convolution arithmetic to reduce the size of an intermediate output (see [Keras documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)).
* Dropout layers help reduce overfitting.

In [ ]:
model = Sequential()
# TODO: Your code here to build, train, and test a cats vs. dogs CNN! (If you run into errors, see the hints above for help debugging!)


In [ ]:
#@title Instructor Solution { display-mode: "form" }
model = Sequential()
#TODO: Your code here to build and train a cats vs. dogs CNN!
#Using architecture from previous exercise - gives around 70% validation accuracy
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))


# initiate RMSprop optimizer
opt = keras.optimizers.legacy.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# Train the CNN and plot accuracy.
# Substituted new dataset names; to_categorical converts to one-hot, as ValueError suggests
history = model.fit(cd_train_inputs, to_categorical(cd_train_labels), \
                    validation_data=(cd_test_inputs, to_categorical(cd_test_labels)), \
                    epochs=30)
plot_acc(history)

print(model.summary())


#Advanced Challenge: Implementing a Famous Architecture for Cats vs. Dogs

Having trouble designing an effective architecture? Try implementing a version of AlexNet, one of the most famous CNNs for image convolution ever. You can find this image and other useful information on this network [here](https://towardsdatascience.com/the-w3h-of-alexnet-vggnet-resnet-and-inception-7baaaecccc96).

![](https://lh4.googleusercontent.com/gFAxn9Z-Y1lgkNy2GfsqjXy1DvSuYF8rvP3CslRvmuoP5SUaJMrEOr24YShU_LwalLpYNJFwpJgcDh9whk9XrMOGQ1ADQ9FY_0saicCVH0jsNPDKOYBcTG4YhbqpbPolW4hZSdUsDQ)

How do we read this diagram?

On the left side, we start with images of dimension 227x227x3 (RGB). We apply a filter composed of 96 kernels of size 11x11, with stride size 4. We end up with data of dimension 55x55x96. We pass through multiple layers of convolution and max pooling as shown, before ending with three dense (fully connected) layers.

Not shown: each layer uses ReLU activation, and we include dropout before the first two dense layers. Make sure to include those!

You'll want to adjust some of these dimensions, for a few reasons: we're starting with 150x150 rather than 227x227 images, ending with 2 labels rather than 1000, and have limited data and memory. Use your knowledge of convolution arithmetic (see CNN slides) and the [Keras documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) to change the stride, kernel, and/or padding.

Use `model.summary()` to understand the dimensions of your data at each step. To speed things up as you're building, you can set the number of epochs to 1.

In [ ]:
model = Sequential()
#TODO: Your code to run, train, and test AlexNet here:


In [ ]:
#@title Instructor Solution { display-mode: "form" }
#The dimensions do not match up perfectly with the diagram
# - there are a few reasonable ways to do this.
#This gives an accuracy of about 75%.

model = Sequential()
#TODO: Your AlexNet code here:
model.add(Conv2D(96, 11, strides = 3))
model.add(Activation('relu'))

model.add(Conv2D(256, 5))
model.add(Activation('relu'))

model.add(MaxPooling2D(2))
model.add(Activation('relu'))

model.add(Conv2D(384, 3, padding = 'same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(2))
model.add(Activation('relu'))

model.add(Conv2D(384, 3, padding = 'same'))
model.add(Activation('relu'))

model.add(Conv2D(256, 3, padding = 'same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(2))
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dropout(0.5))
model.add(Dense(4096))
model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(2))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.legacy.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# Train the CNN and plot accuracy.
# Substituted new dataset names; to_categorical converts to one-hot, as ValueError suggests
history = model.fit(cd_train_inputs, to_categorical(cd_train_labels), \
                    validation_data=(cd_test_inputs, to_categorical(cd_test_labels)), \
                    epochs=10)
plot_acc(history)

print(model.summary())

You might find that even AlexNet isn't working that well for you!

This is because having a good architecture is only half the battle: AlexNet is a complex model designed to learn from millions of images. We're using a small dataset of only 2000 training images, so it's not surprising that our results aren't great. Our model is overfitting: essentially memorizing the few training images, rather than really learning the difference between a cat and a dog. (The advantage is that our model trains quickly.)

To get really good performance, we need more data. If we can't find more, we could use *data augmentation*: inventing new training data by transforming our existing images. You can read more about it at https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html.

#From AlexNet to VGGNet: A Better Choice for Image Classification

AlexNet's architecture may not be complex enough to capture the nuanced differences between cats and dogs, which might result in lower accuracy in classification tasks.

VGGNet, or simply VGG, introduced by the Visual Geometry Group, is a much deeper model with its variants VGG16 and VGG19 having 16 and 19 layers respectively. VGG architecture has smaller, but more number of convolutional filters compared to AlexNet, allowing it to learn more complex features. This depth, along with the use of small 3x3 filters throughout the network, makes it excellent at learning hierarchical features in images, making it more suitable for our cat versus dog classification task.

Moreover, VGG models have been trained on millions of images from the ImageNet
database, which includes a wide variety of animal images, making the learned features more generalized and better suited for our task. Hence, moving from AlexNet to VGG for our specific classification task would be a strategic choice to potentially improve our model's performance.

So let's see how good it is at our task!

In [ ]:
#@title Run this to load images and imports! { display-mode: "form" }
#We have to resize the images to be 240,240 for the vgg net model.

from keras.applications.vgg16 import VGG16
from keras.models import Model
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.layers import BatchNormalization
from keras.optimizers import SGD
from keras.layers import Dense
from keras.layers import Flatten


train_data = train_image_generator.flow_from_directory(batch_size=2000,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(224,224), #(150,150)
                                                           class_mode='binary').next()
val_data = validation_image_generator.flow_from_directory(batch_size=1000,
                                                              directory=validation_dir,
                                                              shuffle=False,
                                                              target_size=(224,224), #(150,150)
                                                              class_mode='binary').next()
cd_train_inputs, cd_train_labels = train_data
cd_test_inputs, cd_test_labels = val_data

It's crucial to note that VGG16 model requires input images to have a size of 224x224 pixels. We've already preprocessed our images to match this requirement, but ensure the model's input shape is set to this dimensionality as well!

Also, you might have noticed that there's another dimension specified in the model's input shape. Wondering what that is? It's the channel dimension. For colored images, it is usually 3, representing the Red, Green, and Blue channels.

Now, let's move onto our model configuration. For the first Dense layer, we will be using the 'relu' (Rectified Linear Unit) activation function. It's a commonly used activation function that introduces non-linearity in our model.

Following that, we'll utilize the 'sigmoid' activation function in our final Dense layer. The sigmoid function squashes the output between the range of 0 and 1, making it suitable for binary classification tasks.

In [ ]:
# Fill in the underlines!

model = VGG16(include_top=False, input_shape=(____,_____,_____))

# mark loaded layers as not trainable
for layer in model.layers:
  layer.trainable = False

flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128, activation='_____', kernel_initializer='he_uniform')(flat1)
output = Dense(2, activation='_______')(class1)

# define new model
model = Model(inputs=model.inputs, outputs=output)

# compile model
opt = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# Train the CNN and plot accuracy.
# Substituted new dataset names; to_categorical converts to one-hot, as ValueError suggests
history = model.fit(cd_train_inputs, to_categorical(cd_train_labels), \
                    validation_data=(cd_test_inputs, to_categorical(cd_test_labels)), \
                    epochs=2)
plot_acc(history)

print(model.summary())

In [ ]:
#@title Instructor Solution { display-mode: "form" }

model = VGG16(include_top=False, input_shape=(224, 224, 3))
	# mark loaded layers as not trainable

for layer in model.layers:
  layer.trainable = False
	# dd new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
output = Dense(2, activation='sigmoid')(class1)
	# define new model
model = Model(inputs=model.inputs, outputs=output)
# compile model
opt = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# Train the CNN and plot accuracy.
# Substituted new dataset names; to_categorical converts to one-hot, as ValueError suggests
history = model.fit(cd_train_inputs, to_categorical(cd_train_labels), \
                    validation_data=(cd_test_inputs, to_categorical(cd_test_labels)), \
                    epochs=2)
plot_acc(history)

print(model.summary())

# You've reached the end!
Moving from AlexNet to VGGNet represents a shift towards more complex and powerful models. Don't feel discouraged if it feels overwhelming – the depth and complexity of VGGNet are part of its strength and why it performs so well.

By experimenting with different hyperparameters, layers, and models you're not only enhancing your problem-solving skills but also your intuition about how neural networks operate.

Remember, even if you don't hit 95% accuracy, the key is to learn from the process and understand why a certain setup works or why it doesn't.

Keep up the fantastic work! Remember, the path to mastering machine learning is a marathon, not a sprint. Keep exploring, keep questioning, and most importantly, have fun with it!

![](https://images.pexels.com/photos/316/black-and-white-animal-dog-pet.jpg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)